In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
import ipywidgets
from IPython.display import display

In [ ]:
team_data = pd.read_csv('/content/team_game_data.csv')
nba_players = pd.read_csv('/content/player_info.csv')


##Team data for the regular season starts from 18 October to 9 April##
team_data['GD'] = pd.to_datetime(team_data['GD'], format = '%d-%b-%y')
start = pd.to_datetime('18-OCT-22', format = '%d-%b-%y')
end = pd.to_datetime('9-APR-23', format = '%d-%b-%y')
temp = team_data[(team_data['GD'] >= start)& (team_data['GD'] <= end)]
regular_season = temp.sort_values('GD')
regular_season.POINTS.value_counts()

#from describe the average league points during the season is 114.685772
regular_season.POINTS.describe()

#most points scored over a regular season game
maximum_points = 0
regular_season.loc[(team_data.POINTS == regular_season.POINTS.max())] #Sacramento Kings 176 points against LA Clippers 175 on 24 February 2023

,GAME_CODE,TEAM_ID,TEAM,OPP_TEAM_ID,OPP,GD,FIELD_GOALS_MADE,FIELD_GOALS_ATT,THREE_POINT_MADE,THREE_POINT_ATT,...,POINTS,TOTAL_REBOUNDS,OFFENSIVE_REBOUNDS,DEFENSIVE_REBOUNDS,ASSISTS,STEALS,BLOCKS,TURNOVERS,PERSONAL_FOULS,OUTCOME
2368,2484911,23,Sacramento Kings,12,Los Angeles Clippers,2023-02-24,65,111,18,41,...,176,43,12,31,33,18,3,15,28,W


In [ ]:
##PLAYER PERFOMANCE OVER THE COURSE OF THE SEASON##
players_in_games = pd.read_csv('/content/player_game_data.csv')

#I'm focusing on the regular season which started on 18 Oct and conclude on 9 Apr
players_in_games['GD'] = pd.to_datetime(players_in_games['GD'], format = '%d-%b-%y')
start = pd.to_datetime('18-OCT-22', format = '%d-%b-%y')
end = pd.to_datetime('9-APR-23', format = '%d-%b-%y')
regular_season_players = players_in_games[(players_in_games['GD'] >= start)& (players_in_games['GD'] <= end)]
player_values = regular_season_players.sort_values('PLAYER')

In [ ]:
def plot_nba_points(Teams):
  #team with the best average points in the league across the season
  team_averages = regular_season.groupby('TEAM').POINTS.mean()

  #Now let's see how they played over the course of the season
  plotted_team = regular_season.loc[regular_season.TEAM == Teams]

  plt.figure(figsize=(7,7))
  plt.bar(plotted_team.GD, plotted_team.POINTS, color = 'blue')
  plt.xlabel('Games')
  plt.ylabel('Points')
  plt.title(f'{ Teams} Points over the regular season')
  plt.legend([f'{Teams} Points'])
  plt.show()



nba_teams = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets','Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','Denver Nuggets','Detroit Pistons','Golden State Warriors','Houston Rockets','Indiana Pacers','Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat','Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans','New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs','Toronto Raptors','Utah Jazz','Washington Wizards']
team_picker = ipywidgets.interactive(plot_nba_points, Teams =nba_teams)
#display(team_picker)

In [ ]:
team_metrics  = ['3 Pointers', '2 Pointers', 'Rebounds', 'Steals and Blocks', 'Free Throws', 'Turnovers','Possession', 'Efficiency']



def team_stats(METRICS):
  #The Philadelphia 76ers have the highest perecentage effociency 3 point shots made but only just ahead of the warriors : 38.691589%
  #The 2 graphs
  teams = regular_season.groupby('TEAM').TEAM.unique().apply(lambda x: x[0])

  if METRICS == '3 Pointers':
    #Who are the best teams by 3point scores over the regular season
    tp = regular_season.groupby('TEAM').THREE_POINT_MADE.mean()
    print(tp.sort_values(ascending = False))


  elif METRICS == '2 Pointers':
    #I will get 2 pointers by subtracting 3 pointers from field goals
    regular_season.groupby('TEAM').FIELD_GOALS_MADE.mean() - regular_season.groupby('TEAM').THREE_POINT_MADE.mean() #Atlanta Hawks lead the way

    #How about the percentage of 2 pointers made as a perecentage
    two_pt_made = regular_season.groupby('TEAM').FIELD_GOALS_MADE.mean()- regular_season.groupby('TEAM').THREE_POINT_MADE.mean()
    total = regular_season.groupby('TEAM').FIELD_GOALS_ATT.mean() - regular_season.groupby('TEAM').THREE_POINT_ATT.mean()
    print(two_pt_made.sort_values(ascending= False))



  elif METRICS == 'Rebounds':
    #best rebounding team
    rebounds = regular_season.groupby('TEAM').TOTAL_REBOUNDS.mean() #The Milwaukee Bucks take the crown with 48.634146 rebounds per game
    print(rebounds.sort_values(ascending=False))



  elif METRICS == 'Steals and Blocks':
    #team with the most steals, and blocks
    steals = regular_season.groupby('TEAM').STEALS.mean() + regular_season.groupby('TEAM').BLOCKS.mean()
    print(steals.sort_values(ascending = False))





  elif METRICS == 'Free Throws':
    ##best freethrow by total per game and percentage##
    ft_made = regular_season.groupby('TEAM').FREE_THROWS_MADE.mean()
    ft_percentage = team_data.groupby('TEAM').FREE_THROWS_ATT.mean()
    print(ft_made.sort_values(ascending = False))



  elif METRICS == 'Turnovers':
    #best turnover team
    turnovers = regular_season.groupby('TEAM').TURNOVERS.mean()
    print(turnovers.sort_values(ascending=False))


  elif METRICS == 'Possession':
    regular_season['POSSESSION'] = 0.96*(regular_season['FIELD_GOALS_ATT'] + regular_season['TURNOVERS'] + 0.44*(regular_season['FREE_THROWS_ATT'])-regular_season['OFFENSIVE_REBOUNDS'])
    possessions = regular_season.groupby('TEAM').POSSESSION.mean()
    print(possessions.sort_values(ascending =False).head())

  elif METRICS == 'Efficiency':
    regular_season['EFFICIENCY'] = 100*(regular_season['POINTS']/regular_season['POSSESSION'])
    efficiency = regular_season.groupby('TEAM').EFFICIENCY.mean()
    print(efficiency.sort_values(ascending = False).head())

#NOW THE WIDGET
stats_and_metrics = ipywidgets.interactive(team_stats, METRICS = team_metrics)
#display(stats_and_metrics)
widget_box = ipywidgets.HBox([ stats_and_metrics, team_picker])
display(widget_box)



In [ ]:
#PLAYER WIDGETS
def player_metrics(STATS):
  if STATS == 'Points':
    #Most Points per game
    games = player_values.groupby('PLAYER').GAMES.sum()
    player_points = player_values.groupby('PLAYER').POINTS.sum()
    ppg = player_points/games
    print(ppg.sort_values(ascending= False)[0:20])

  elif STATS == '3 Pointers':
    #The best 3 point shooters
    three_pt_made = player_values.groupby('PLAYER').THREE_POINT_MADE.sum()/player_values.groupby('PLAYER').GAMES.sum()
    three_pt_attempted = player_values.groupby('PLAYER').THREE_POINT_ATT.sum()
    print(three_pt_made.sort_values(ascending = False)[0:20])

  elif STATS == 'Free Throws':
    #Best at FT
    ft_made = player_values.groupby('PLAYER').FREE_THROWS_MADE.sum()
    ft_per_game = player_values.groupby('PLAYER').FREE_THROWS_MADE.sum()/player_values.groupby('PLAYER').GAMES.sum()
    print(ft_per_game.sort_values(ascending = False)[0:20])

  elif STATS == 'Steals and Blocks':
    #Who makes the most steals and Blocks
    s_n_b_made = player_values.groupby('PLAYER').STEALS.sum() + player_values.groupby('PLAYER').BLOCKS.sum()
    s_n_b_pg = (player_values.groupby('PLAYER').STEALS.sum() + player_values.groupby('PLAYER').BLOCKS.sum())/player_values.groupby('PLAYER').GAMES.sum()
    print(s_n_b_pg.sort_values(ascending = False)[0:20])

  elif STATS == 'Rebounds':
    #Best at Rebounds made
    rebounds_made = player_values.groupby('PLAYER').TOTAL_REBOUNDS.sum()
    rebounds_per_game = player_values.groupby('PLAYER').TOTAL_REBOUNDS.sum()/player_values.groupby('PLAYER').GAMES.sum()
    print(rebounds_per_game.sort_values(ascending = False)[0:20])

  elif STATS == 'Turnovers':
    #Best at Turnovers made
    turnovers_made = player_values.groupby('PLAYER').TURNOVERS.sum()
    turnovers_per_game = player_values.groupby('PLAYER').TURNOVERS.sum()/player_values.groupby('PLAYER').GAMES.sum()
    print(turnovers_per_game.sort_values(ascending =False)[0:20])

  elif STATS == 'Minutes':
    #Most minutes played
    mins = player_values.groupby('PLAYER').MINUTES.sum()
    mins.sort_values(ascending=False)
    mins_per_game = player_values.groupby('PLAYER').MINUTES.sum()/player_values.groupby('PLAYER').GAMES.sum()
    print(mins_per_game.sort_values(ascending=False)[0:20])

  elif STATS == 'Triple Doubles':
    #Triple Doubles
    for index, row in player_values.iterrows():
      if row['POINTS'] >=10 and row['TOTAL_REBOUNDS'] >= 10 and row['ASSISTS'] >= 10:
        player_values.at[index,'TRIPLE_DOUBLE'] = 1
      else:
        player_values.at[index, 'TRIPLE_DOUBLE'] = 0

    td_made = player_values.groupby('PLAYER').TRIPLE_DOUBLE.sum()
    td_per_game = player_values.groupby('PLAYER').TRIPLE_DOUBLE.sum()/player_values.groupby('PLAYER').GAMES.sum()
    print(td_per_game.sort_values(ascending = False)[0:20])




p_metrics = ['Points','3 Pointers','Rebounds', 'Steals and Blocks', 'Free Throws', 'Turnovers', 'Minutes', 'Triple Doubles']
player_stats = ipywidgets.interactive(player_metrics,STATS = p_metrics)

display(player_stats)

interactive(children=(Dropdown(description='STATS', options=('Points', '3 Pointers', 'Rebounds', 'Steals and B…

In [ ]:
def specific_teams(Team):
    unique_players = player_values.loc[player_values['TEAM'] == Team, 'PLAYER'].unique()
    player_dropdown = ipywidgets.Dropdown(options=unique_players)
    player_dropdown.observe(specific_player, names='value')
    display(player_dropdown)

def specific_player(change):
    player_name = change.new
    #turnovers_made = player_values[player_values['PLAYER'] == player_name]['TURNOVERS'].sum()
    #turnovers_per_game = turnovers_made / player_values[player_values['PLAYER'] == player_name]['GAMES'].sum()
    games = player_values[player_values['PLAYER']==player_name]['GAMES'].sum()
    player_points = player_values[player_values['PLAYER']==player_name]['POINTS'].sum()
    ppg = player_points/player_values[player_values['PLAYER']==player_name]['GAMES'].sum()
    #print(f"Turnovers per game for {player_name}: {turnovers_per_game:.2f}")
    print(f"Points per game for {player_name}: {ppg:.2f}")


nba_teams = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets','Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','Denver Nuggets','Detroit Pistons','Golden State Warriors','Houston Rockets','Indiana Pacers','Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat','Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans','New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs','Toronto Raptors','Utah Jazz','Washington Wizards']


teams1 = ipywidgets.interactive(specific_teams, Team=nba_teams)
display(teams1)

interactive(children=(Dropdown(description='Team', options=('Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets'…

Points per game for Stephen Curry: 29.43
